In [19]:
from multiprocessing import Process,Queue
# from multiprocessing import JoinableQueue as Queue
# import multiprocessing.JoinableQueue as Queue
import os
import sys

INF = 1e9
SourceNode = 1
data_file = "wiki"
class HyperNode:
    def __init__(self,n_id,edges):
        self.n_id = n_id
        self.dist = INF 
        self.edges = edges
        
    def AddEdge(self,edge):
        self.edges.append(edge)
        
    def GetHyperEdges(self):
        return self.edges
    
    def GetValue(self):
        return self.dist
    
                    
class HyperEdge:
    def __init__(self,e_id,nodes):
        self.e_id = e_id
        self.nodes = nodes
        
    def AddNode(self,node):
        self.nodes.append(node)
    
    def GetHyperNodes(self):
        return self.nodes
    
    def getValue(self):
        return 1

    

In [31]:
class Master:
    def __init__(self,p):
        self.p = p
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.Worker = []
        
        self.thread_pool = []
        self.m2sPipe = []
        self.s2mPipe = []
        self.recvBuf = [Queue() for i in range(p)]
        
        self.result = {}
        for i in range(p) :
            self.Worker.append(Worker(i))
            self.m2sPipe.append(Queue())
            self.s2mPipe.append(Queue())
            
        for worker in self.Worker:
            w_id = worker.w_id
            self.thread_pool.append(Process(target= worker.run,args=(self.m2sPipe[w_id],self.s2mPipe[w_id],self.recvBuf)))

    def terminal(self):
        for pipe in self.m2sPipe:
            pipe.close()
        for pipe in self.s2mPipe:
            pipe.close()
            
    def load_data(self):
        with open("./test_data/"+data_file+"/partition_method.txt") as file :  # (n_id,p_id)
            for line in file:
                v_id,par_id = line[0:-1].split(" ")
                v_id,par_id = int(v_id),int(par_id)
                self.Part[v_id] = par_id
                self.Node[v_id] = []
                    
        with open("./test_data/"+data_file+"/vertex_info.txt") as file:  # v_id, e1_id, e2_id, ...
            for line in file:
                data = line[0:-1].split(" ")
                v_id = int(data[0])
                self.Node[v_id] = [int(i) for i in data[1:]]
                    
        with open("./test_data/"+data_file+"/edge_info.txt") as file:   # e_id, v1_id, v2_id,...
            for line in file:
                data = line[0:-1].split(" ")
                e_id = int(data[0])
                self.Edge[e_id] = [int(i) for i in data[1:]]

    def sync(self):
        return [sign.get()  for sign in self.s2mPipe]
    
    def broadcast(self,signal=0):
#         sys.stdout.write("signal "+str(signal)+"\n")
        for m2sPipe in self.m2sPipe:
                m2sPipe.put(signal)
            
    def distribute_node(self):
        for n_id,p_id in self.Part.items():
            self.m2sPipe[p_id].put((n_id,self.Node[n_id]))
        for m2sPipe in self.m2sPipe:
            m2sPipe.put((-1,[-1]))

        self.sync()
#         self.Worker[w_id].Node[n_id] = self.Node[n_id]
    
    def distribute_edge(self):
        for e_id,node in self.Edge.items():
            check = {}
            for n_id in node :
                p_id = self.Part[n_id]
                if len(check) == self.p : break
                if p_id not in check:
                    self.m2sPipe[p_id].put((e_id,node))
                    check[p_id] = 1
        for m2sPipe in self.m2sPipe:
            m2sPipe.put((-1,[-1]))
            
        self.sync()
    
    def distribute_part(self):
        for worker in self.Worker:
            for item in self.Part.items():
                self.m2sPipe[worker.w_id].put(item)
            self.m2sPipe[worker.w_id].put((-1,-1))
            
            
        self.sync()
        
    def distribute(self):
        self.distribute_part()
        self.distribute_node()
        self.distribute_edge()
        
    def get_result(self):
        cnt = 0

        for pipe in self.s2mPipe: 
            while True:
                v_id,dist  = pipe.get()
#                 sys.stdout.write("update dist:"+ str(v_id)+" "+str(dist))
                if v_id == -1: 
                    break
                self.result[v_id] = dist
        sys.stdout.write("get result !\n")
        
        
        

    def run(self):
        for thread in self.thread_pool:
            thread.start()
        sys.stdout.write("All thread start"+"\n")
            
        self.distribute()
        sys.stdout.write("init data distribute over"+"\n")
        
        msg_num = 1
        turn = 0
        
        source_id = 1 # init status
        self.recvBuf[self.Part[source_id]].put((source_id,0))
        for buf in self.recvBuf: 
            buf.put((-1,-1))
        sys.stdout.write("master edge num:"+str(len(self.Node[source_id])))
        
        while msg_num != 0:
            sys.stdout.write("beign turn:"+str(turn)+"\n")
            turn += 1
    
            self.broadcast(0) # begin recevie buffer 
            msg = self.sync() # wait finish recevie buffer
            sys.stdout.write("receive buffer info ok"+"\n")
            
            self.broadcast(0) # begin compute
            msg = self.sync() # wait finish compute
            sys.stdout.write("compute ok"+"\n")
            msg_num = sum([item[0] for item in msg])
            cro_num = sum([item[1] for item in msg])
            while True:
                if sum([buf.qsize() for buf in self.recvBuf]) >= cro_num:
                    break
                
            sys.stdout.write(str(turn)+" "+str(msg_num)+"\n")
            
            for buf in self.recvBuf:# add recevie buffer finished sign
                buf.put((-1,-1))
        
#         sys.stdout.write("terminal "+"\n")
        self.broadcast(-1) # off worker
#         self.sync()
#         sys.stdout.write("wokers shut down"+"\n")
            
        self.get_result()
        self.terminal()
        
        sys.stdout.write("finished"+"\n")
        
            
          
class Worker:
    def __init__(self,w_id):
        self.w_id = w_id
        self.Node = {}
        self.Edge = {}
        self.Part = {}
        self.msg = {}
        self.sendBuf = []
        self.recvBuf = []
        filepath = "./pipeline/"+str(self.w_id)
        open(filepath,"w")
        
    def cal_node(self,node):
        msgs = self.msg[node.n_id]
        mindist = INF 
        mindist = min(msgs)
        sendMessage = []
        if mindist < node.dist : 
            node.dist = mindist
            
            for e_id in node.GetHyperEdges():
#                 print("debug:",self.w_id," ",e_id)
                hyperEdge = self.Edge[e_id]
                for n_id in hyperEdge.GetHyperNodes():
                    sendMessage.append((n_id,mindist+hyperEdge.getValue()))
#                     SendMessageTo(self.v_id,target,mindist+hyperEdge.getValue())
        return sendMessage
    
    def receive_node(self,m2sPipe,s2mPipe):
        while True:
            v_id,edge = m2sPipe.get()
            
            if v_id == -1 : break
            if v_id in self.Node : 
                continue
            self.Node[v_id] = HyperNode(v_id,edge)
#             if v_id == 2:
#                 print("server ",self.w_id, " edge num:",len(self.Node[v_id].GetHyperEdges())," ",len(edge))
        s2mPipe.put(self.w_id)
#         print(self.w_id," node_num:",len(self.Node))
        
    def receive_edge(self,m2sPipe,s2mPipe):
        while True:
            e_id,node = m2sPipe.get()
            if e_id == -1 : break
            if e_id in self.Edge : 
                continue
            self.Edge[e_id] = HyperEdge(e_id,node)
        s2mPipe.put(self.w_id)
#         print(self.w_id," edge_num:",len(self.Edge))
        
        
    def receive_part(self,m2sPipe,s2mPipe):
        while True:
            v_id,p_id = m2sPipe.get()
            if v_id == -1 : break
            self.Part[v_id] = p_id
#             if len(self.Part)%10000 == 0 :
#                 print(self.w_id," ",len(self.Part))
        s2mPipe.put(self.w_id)
            

    def run(self,m2sPipe,s2mPipe,sendBuf):
        self.receive_part(m2sPipe,s2mPipe)
        self.receive_node(m2sPipe,s2mPipe)
        self.receive_edge(m2sPipe,s2mPipe)
        
        while True:
            sig = m2sPipe.get()
#             sys.stdout.write("w_id:"+str(self.w_id)+" sign:"+str(sig)+"\n")
            if sig == -1 : 
                break
            self.receiveBuf(s2mPipe,sendBuf)
            
            m2sPipe.get()
            self.compute(s2mPipe,sendBuf) 
            
        sys.stdout.write("w_id:"+str(self.w_id)+" test "+str(len(self.Node))+"\n")
        for node in self.Node.values():
            s2mPipe.put((node.n_id,node.dist))
        sys.stdout.write("w_id:"+str(self.w_id)+" send result"+"\n")
        s2mPipe.put((-1,-1))
        return 
        
    def receiveBuf(self,s2mPipe,Buffer):
        self.msg = {}
        while True:
            tar,value = Buffer[self.w_id].get()
            if tar == -1 : break
                
            if tar not in self.msg :
                self.msg[tar] = []
            self.msg[tar].append(value)
            
        for tar,value in self.recvBuf:
            if tar not in self.msg :
                self.msg[tar] = []
            self.msg[tar].append(value)
        self.recvBuf = []
        
#         print("TEST:",self.w_id," ",len(self.msg))
        
        s2mPipe.put(self.w_id)
        
    def compute(self,s2mPipe,Buffer):
        msg_cnt = 0
        cross_cnt = 0
        debug_cnt = 100000
        for n_id,node in self.Node.items():
            if n_id not in self.msg : continue
            msg = self.cal_node(node) # compute and get msg info
            msg_cnt += len(msg)
            for tar,value in msg:  # msg transfer judge
                debug_cnt = min(debug_cnt,value)
                if self.Part[tar] == self.w_id:
                    self.recvBuf.append((tar,value))
                else : 
                    Buffer[self.Part[tar]].put((tar,value))
                    cross_cnt += 1
#         print("worker msg:",self.w_id," ",msg_cnt)
#         sys.stdout.write("worker msg:"+str(self.w_id)+" "+str(msg_cnt)+"\n")
        sys.stdout.write("w_id:"+str(self.w_id)+" mini dist:"+str(debug_cnt)+"\n")
        s2mPipe.put((msg_cnt,cross_cnt))
        
master = Master(4)
master.load_data()
master.run()


All thread start
w_id:3 mini dist:100000
w_id:0 mini dist:100000
w_id:1 mini dist:100000
w_id:2 mini dist:1
init data distribute over
master edge num:11beign turn:0
receive buffer info ok
compute ok
1 1239
beign turn:1
receive buffer info ok
w_id:3 mini dist:2
w_id:2 mini dist:2
w_id:1 mini dist:2
w_id:0 mini dist:2
compute ok
2 5797283
beign turn:2
receive buffer info ok
w_id:3 mini dist:3
w_id:2 mini dist:3
w_id:1 mini dist:3
w_id:0 mini dist:3
compute ok
3 15201811
beign turn:3
w_id:2 mini dist:4
w_id:1 mini dist:100000
w_id:0 mini dist:100000
w_id:3 mini dist:3
w_id:1 mini dist:100000
w_id:2 mini dist:100000
w_id:0 mini dist:100000
w_id:3 mini dist:3
w_id:2 mini dist:100000
w_id:1 mini dist:100000
w_id:0 mini dist:100000
w_id:3 mini dist:3
w_id:1 mini dist:100000
w_id:2 mini dist:100000
w_id:0 mini dist:100000
w_id:3 mini dist:3
w_id:1 mini dist:100000
w_id:2 mini dist:100000
w_id:0 mini dist:100000
w_id:3 mini dist:3
receive buffer info ok
compute ok
4 29867
beign turn:4
receive b

In [32]:
dic = {}
for key,value in master.result.items():
    if value not in dic : 
        dic[value] = 0
    dic[value] += 1
for k,v in dic.items():
    print(k,":",v)

2 : 3720
1 : 804
1000000000.0 : 11
3 : 35
0 : 1
